In [6]:
from qiskit import QuantumCircuit
from ClassicalCircuitExercise import ClassicalCircuit


In [7]:
def convert_step_1(self, quantumCircuit):
    for gate in self.gates:
        match gate[1]:
            case "and":
                a, b, c = gate[0], gate[2], gate[3]
                quantumCircuit.ccx(b, c, a)
            case "not":
                a, b = gate[0], gate[2]
                quantumCircuit.x(a)
                quantumCircuit.cx(b, a)
            case "or":
                a, b, c = gate[0], gate[2], gate[3]
                quantumCircuit.x(b)
                quantumCircuit.x(c)
                quantumCircuit.ccx(b, c, a)
                quantumCircuit.x(b)
                quantumCircuit.x(c)
                quantumCircuit.x(a)
            case "xor":
                # bonus, cnot kinda is xor
                a, b, c = gate[0], gate[2], gate[3]
                quantumCircuit.cx(b, a)
                quantumCircuit.cx(c, a)
            case "nand":
                # bonus, toffoli is and so not + toffoli = nand
                a, b, c = gate[0], gate[2], gate[3]
                quantumCircuit.ccx(b, c, a)
                quantumCircuit.x(a)
        quantumCircuit.barrier()

ClassicalCircuit.convert_step_1 = convert_step_1

In [8]:
def convert_step_2(self, quantumCircuit):
    for gate in reversed(self.gates):
        match gate[1]:
            case "and":
                a, b, c = gate[0], gate[2], gate[3]
                quantumCircuit.ccx(b, c, a)
            case "not":
                a, b = gate[0], gate[2]
                quantumCircuit.cx(b, a)
                quantumCircuit.x(a)
            case "or":
                a, b, c = gate[0], gate[2], gate[3]
                quantumCircuit.x(a)
                quantumCircuit.x(c)
                quantumCircuit.x(b)
                quantumCircuit.ccx(b, c, a)
                quantumCircuit.x(c)
                quantumCircuit.x(b)
            case "xor":
                # bonus, reversed
                a, b, c = gate[0], gate[2], gate[3]
                quantumCircuit.cx(c, a)
                quantumCircuit.cx(b, a)
            case "nand":
                # bonus, reversed
                a, b, c = gate[0], gate[2], gate[3]
                quantumCircuit.x(a)
                quantumCircuit.ccx(b, c, a)
        quantumCircuit.barrier()

ClassicalCircuit.convert_step_2 = convert_step_2

In [9]:
def convert(self, quantumCircuit):
    self.convert_step_1(quantumCircuit)

    n_wires = self.n_inputs + self.n_outputs + self.n_internal

    for i, output_gate in enumerate(self.output_gates):
        a = output_gate
        a_prime = n_wires + i
        quantumCircuit.cx(a, a_prime)

    quantumCircuit.barrier()
    self.convert_step_2(quantumCircuit)

ClassicalCircuit.convert = convert

In [10]:
cc = ClassicalCircuit("circuit.txt")
cc.print()


n_inputs: 3
n_outputs: 2
n_internal: 2
Input Gates: [0, 1, 2]
Output Gates: [3, 4]
Internal Gates: [5, 6]
Gates: 
[5, 'and', 0, 1]
[3, 'not', 5]
[6, 'not', 2]
[4, 'and', 5, 6]



In [11]:

n_wires = cc.n_inputs + cc.n_outputs + cc.n_internal
qc = QuantumCircuit(n_wires, 0)
cc.convert_step_1(qc)
print(qc)
print()

           ░            ░            ░       ░ 
q_0: ──■───░────────────░────────────░───────░─
       │   ░            ░            ░       ░ 
q_1: ──■───░────────────░────────────░───────░─
       │   ░            ░            ░       ░ 
q_2: ──┼───░────────────░────────■───░───────░─
       │   ░ ┌───┐┌───┐ ░        │   ░       ░ 
q_3: ──┼───░─┤ X ├┤ X ├─░────────┼───░───────░─
       │   ░ └───┘└─┬─┘ ░        │   ░ ┌───┐ ░ 
q_4: ──┼───░────────┼───░────────┼───░─┤ X ├─░─
     ┌─┴─┐ ░        │   ░        │   ░ └─┬─┘ ░ 
q_5: ┤ X ├─░────────■───░────────┼───░───■───░─
     └───┘ ░            ░ ┌───┐┌─┴─┐ ░   │   ░ 
q_6: ──────░────────────░─┤ X ├┤ X ├─░───■───░─
           ░            ░ └───┘└───┘ ░       ░ 



In [12]:
n_wires = cc.n_inputs + cc.n_outputs + cc.n_internal
qc = QuantumCircuit(n_wires, 0)
cc.convert_step_2(qc)
print(qc)
print()

           ░            ░            ░       ░ 
q_0: ──────░────────────░────────────░───■───░─
           ░            ░            ░   │   ░ 
q_1: ──────░────────────░────────────░───■───░─
           ░            ░            ░   │   ░ 
q_2: ──────░───■────────░────────────░───┼───░─
           ░   │        ░ ┌───┐┌───┐ ░   │   ░ 
q_3: ──────░───┼────────░─┤ X ├┤ X ├─░───┼───░─
     ┌───┐ ░   │        ░ └─┬─┘└───┘ ░   │   ░ 
q_4: ┤ X ├─░───┼────────░───┼────────░───┼───░─
     └─┬─┘ ░   │        ░   │        ░ ┌─┴─┐ ░ 
q_5: ──■───░───┼────────░───■────────░─┤ X ├─░─
       │   ░ ┌─┴─┐┌───┐ ░            ░ └───┘ ░ 
q_6: ──■───░─┤ X ├┤ X ├─░────────────░───────░─
           ░ └───┘└───┘ ░            ░       ░ 



In [13]:

n_wires = cc.n_inputs + 2 * cc.n_outputs + cc.n_internal
qc = QuantumCircuit(n_wires, 0)
cc.convert(qc)
print(qc)
print()

           ░            ░            ░       ░            ░       ░           »
q_0: ──■───░────────────░────────────░───────░────────────░───────░───────────»
       │   ░            ░            ░       ░            ░       ░           »
q_1: ──■───░────────────░────────────░───────░────────────░───────░───────────»
       │   ░            ░            ░       ░            ░       ░           »
q_2: ──┼───░────────────░────────■───░───────░────────────░───────░───■───────»
       │   ░ ┌───┐┌───┐ ░        │   ░       ░            ░       ░   │       »
q_3: ──┼───░─┤ X ├┤ X ├─░────────┼───░───────░───■────────░───────░───┼───────»
       │   ░ └───┘└─┬─┘ ░        │   ░ ┌───┐ ░   │        ░ ┌───┐ ░   │       »
q_4: ──┼───░────────┼───░────────┼───░─┤ X ├─░───┼────■───░─┤ X ├─░───┼───────»
     ┌─┴─┐ ░        │   ░        │   ░ └─┬─┘ ░   │    │   ░ └─┬─┘ ░   │       »
q_5: ┤ X ├─░────────■───░────────┼───░───■───░───┼────┼───░───■───░───┼───────»
     └───┘ ░            ░ ┌───┐┌─┴─┐ ░  